In [54]:
import subprocess

def blame(file):
    print(file)
    return subprocess.run(["git", "blame", "-p", file],
       capture_output=True,
       cwd=repo_root
    ).stdout.decode("utf8")

def git_ls_files(repo_root, dir=None):
    args = ["git", "ls-files"]
    if dir is not None:
        args.append(dir) 
        
    p = subprocess.run(args,
       capture_output=True,
       cwd=repo_root
    )
    
    if p.returncode != 0:
        return p.stderr.decode("utf8")
    return p.stdout.decode("utf8").split("\n")


In [198]:
import re, pathlib, os.path as path

limit = 3
root_dir = "/home/bwhite/Projects/bryanchriswhite"
repo_names = [
    "storj",
]
author_regex = re.compile("\S+\s+\(([\w ]+)\s*?\d{4,}")

header_re = re.compile("\w{,40} \d+ (\d+)(:? (\d+))?")
author_re = re.compile("author (.+)$")

author_line_counts = {}

for name in repo_names:
    repo_root = path.join(root_dir, name)
    files = git_ls_files(repo_root=repo_root)
    for i, f in enumerate(files):
        if f == "go.mod":
        # if f != "" and i < limit:
            last_headers = [1, 1]
            last_authors = [None, None]
            
            # print(author_line_counts)
            
            # print(f)
            blame_lines = blame(path.join(repo_root, f)).split("\n")
            for j, line in enumerate(blame_lines):
                if line == "":
                    continue

                # print(line)
                header_match = header_re.match(line)
                if header_match is not None:
                    print("header match (%s)" % j)
                    header_new_ln = header_match.group(1)
                    # print(line)
                    # print("header match g: %s" % header_match_g)
                    # previous_header_ln = last_header_ln
                    # last_header_ln = header_match_g
                    last_headers[1] = int(header_new_ln)
                    # print(str.format("ln: {}; count: {}", ln, count))
                    continue

                author_match = author_re.match(line)
                if author_match is not None:
                    # last_headers[0] = int(header_new_ln)
                    
                    # print(author_line_counts)
                    
                    diff = last_headers[1] - last_headers[0]
                    
                    last_author_match = author_match
                    last_author = last_authors[0] = last_authors[1]
                    author = last_authors[1] = author_match.group(1)
                    # author = last_author = author_match.group(1)
                    print("%s; %s; diff: %s" % (last_headers[0], last_headers[1], diff))
                    print("new author (%s | %s -> %s)" % (j, last_author, author))
                    # print(author_line_counts)
                    
                    last_headers[0] = last_headers[1]

                    if last_author not in author_line_counts:
                        # print(author_line_counts)
                        author_line_counts[last_author] = diff
                        print(author_line_counts)
                        continue
                        
                    # print(author_line_counts)
                    author_line_counts[last_author] += diff
                    print(author_line_counts)
                    

/home/bwhite/Projects/bryanchriswhite/storj/go.mod
header match (0)
1; 1; diff: 0
new author (1 | None -> Alexander Leitner)
{None: 0}
header match (13)
2; 2; diff: 0
new author (14 | Alexander Leitner -> Kaloyan Raev)
{None: 0, 'Alexander Leitner': 0}
header match (26)
3; 3; diff: 0
new author (27 | Kaloyan Raev -> Jess G)
{None: 0, 'Alexander Leitner': 0, 'Kaloyan Raev': 0}
header match (39)
4; 4; diff: 0
new author (40 | Jess G -> Egon Elbre)
{None: 0, 'Alexander Leitner': 0, 'Kaloyan Raev': 0, 'Jess G': 0}
header match (52)
header match (54)
6; 6; diff: 0
new author (55 | Egon Elbre -> Jeff Wendling)
{None: 0, 'Alexander Leitner': 0, 'Kaloyan Raev': 0, 'Jess G': 0, 'Egon Elbre': 0}
header match (67)
7; 7; diff: 0
new author (68 | Jeff Wendling -> Egon Elbre)
{None: 0, 'Alexander Leitner': 0, 'Kaloyan Raev': 0, 'Jess G': 0, 'Egon Elbre': 0, 'Jeff Wendling': 0}
header match (80)
8; 8; diff: 0
new author (81 | Egon Elbre -> Egon Elbre)
{None: 0, 'Alexander Leitner': 0, 'Kaloyan Raev':

In [76]:
author_line_counts
                    

{}

In [24]:
import pickle
# from github import Github

print(root_dir)
pickle_path = path.join(root_dir, "storj_repos.pickle")
storj_repos_file = open(pickle_path, "wb")
# apikey = open("github.apikey")
#                
# g = Github()
# org = g.get_organization("storj")
# repos = org.get_repos()
# page = repos.get_page(0)

print(pickle_path)
storj_repos_file.write(b"testing 123")
storj_repos_file.close()
# pickle.dump(page, storj_repos_file)

/home/bwhite/Projects
/home/bwhite/Projects/storj_repos.pickle


In [18]:
page = pickle.load(storj_repos_file)

[(r.name, r.created_at) for r in page]

UnsupportedOperation: read